##### Reinforcement Learning and Decision Making &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Homework #5

# &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Bar Brawl

## Description

You are the proprietor of an establishment that sells beverages of an unspecified, but delicious, nature. The establishment is frequented by a set $P$ of patrons.  One of the patrons is the instigator and another is the peacemaker.

On any given evening, a subset $S \subseteq P$ is present at the establishment. If the instigator is in $S$ but the peacemaker is not in $S$, then a fight will break out. If the instigator is not in $S$ or if the peacemaker is in $S$, then no fight will occur.

Your goal is to learn to predict whether a fight will break out among the subset of
patrons present on a given evening, without initially knowing the identity of
the instigator or the peacemaker.

## Procedure

Develop a KWIK learner for this problem (see Li, Littman, and Walsh 2008).  Your learner will be presented with $S$, the patrons at the establishment, and the outcome (fight or no fight) of that evening.
Your learner will attempt to predict whether a fight will break out, or indicate that
it doesn't know, and should be capable of learning from the true outcome for the evening.

For each problem, the following input will be given:

-   `at_establishment`: a Boolean two-dimensional array whose rows
    represent distinct evenings and whose columns represent distinct patrons.
    Each entry specifies if a particular patron is present
    at the establishment on a particular evening:
    a $1$ means present and a $0$ means absent.
    
-   `fight_occurred`: a Boolean vector whose entries are the 
    outcomes (a $1$ means ``FIGHT`` and a $0$ means ``NO FIGHT``) for that
    particular evening.


Specifically:

-   For each episode (evening), you should present your learner with the next row of `at_establishment` and the corresponding row of `fight_occurred`.
-   If your learner returns a $1$ (for ``FIGHT``) or a $0$ (for ``NO FIGHT``),
    you may continue on to the next episode.
-   If your learner returns a $2$ (for ``I DON’T KNOW``), then you should
    present the pair (`at_establishment`, `fight_occurred`) to you learner
    to learn from.

You will return a string of integers corresponding to the returned values of each episode.

The test case will be considered successful if no wrong answers are returned
**and** the number of "I DON'T KNOW"s does not exceed the maximum 
allowed by the autograder.

## Resources

The concepts explored in this homework are covered by:

-   'Knows what it knows: A framework for self-aware learning', Li, Littman, Walsh 2008

## Submission

-   The due date is indicated on the Canvas page for this assignment.
    Make sure you have your timezone in Canvas set to ensure the
    deadline is accurate.

-   Submit your finished notebook on Gradescope. Your grade is based on
    a set of hidden test cases. You will have unlimited submissions.
    By default, the last score is kept.  You can also set a particular
    submission as active in the submission history, in which case that
    submission will determine your grade.

-   Use the template below to implement your code. We have also provided
    some test cases for you. If your code passes the given test cases,
    it will run (though possibly not pass all the tests) on Gradescope.

-   Gradescope is using *python 3.6.x* and *numpy==1.18.0*, and you can
    use any core library (i.e., anything in the Python standard library).
    No other library can be used.  Also, make sure the name of your
    notebook matches the name of the provided notebook.  Gradescope times
    out after 10 minutes.

In [7]:
################
# DO NOT REMOVE
# Versions
# numpy==1.18.0
################
import numpy as np
from itertools import permutations

class Agent(object):
    def __init__(self):
        self.hypotheses = []
        self.count = 0
    def handle_learn_vs_pred(self, evening_patrons, fight_exists):
        if sum(evening_patrons) == self.count:
            return self.hypotheses, 0

        fight_scenarios = []
        for hypothesis in self.hypotheses:
            if evening_patrons[hypothesis[1]] != 1 and evening_patrons[hypothesis[0]] == 1:
                fight_scenarios.append(1)
            else:
                fight_scenarios.append(0)

        if sum(fight_scenarios) == 0 or sum(fight_scenarios) == len(fight_scenarios):
            return self.hypotheses, fight_scenarios[0]

        else:
            scenarios_to_delete = []
            for i, x in enumerate(fight_scenarios):
                if x == 1-fight_exists:
                    scenarios_to_delete.append(i)
            for j in sorted(scenarios_to_delete, reverse=True):
                self.hypotheses.pop(j)
            return self.hypotheses, 2

    def solve(self, at_establishment, fight_occurred):
        self.count = len(at_establishment[0])
        patrons = list(range(self.count))
        self.hypotheses = list(permutations(patrons, 2))
        pred_results = []
        pred_string_results = pred_results.copy()

        for evening in range(len(at_establishment)):
            new_hypotheses, pred = self.handle_learn_vs_pred(at_establishment[evening], fight_occurred[evening])
            pred_results.append(pred)
            self.hypotheses = new_hypotheses
            if pred == 1:
                pred_string_results.append('1')
            elif pred == 2:
                pred_string_results.append('2')
            else:
                pred_string_results.append('0')
        return ''.join(pred_string_results)

In [8]:

## DO NOT MODIFY THIS CODE.  This code will ensure that you submission is correct 
## and will work proberly with the autograder

import unittest


class TestBarBrawl(unittest.TestCase):
    @classmethod
    def setUpClass(cls):
        cls.agent = Agent()

    def test_case_1(self):
        np.testing.assert_equal(
            self.agent.solve(
                [[1,1], [1,0], [0,1], [1,1], [0,0], [1,0], [1,1]],
                [0, 1, 0, 0, 0, 1, 0]
            ),
            '0200010'
        )

    def test_case_2(self):
        np.testing.assert_equal(
             self.agent.solve(
                [[1,0,0,0,],[0,1,0,0],[0,1,1,1],[0,1,1,1],[0,1,1,1],[0,0,0,1],[1,1,1,1],[1,1,1,1],[1,1,1,1],[1,1,1,1],[0,1,1,1],[1,1,1,1],[0,1,1,1],[0,1,1,1],[1,1,1,1],[0,1,1,1],[0,1,1,1],[1,1,1,1],[0,1,1,1],[0,1,1,1],[0,0,0,1],[0,0,0,1],[1,1,1,1],[0,1,1,1],[0,1,1,1],[0,0,1,1],[0,1,1,1],[0,0,0,1],[0,0,1,1],[0,1,1,1],[0,0,1,1],[1,1,1,1]],
                [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0]
            ),
            '22200200000000000000000002001010'
        )

    def test_case_3(self):
        np.testing.assert_equal(
             self.agent.solve(
                [[1,0,1],[1,0,1],[1,1,1],[1,1,1],[1,1,1],[0,1,1],[0,0,1],[0,1,1],[1,1,1],[1,1,1],[1,1,1],[1,1,1],[0,1,1],[1,0,1],[1,1,1],[1,0,1],[1,0,1],[1,1,1]],
                [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
            ),
            '200002000000000000'
        )
        
unittest.main(argv=[''], verbosity=2, exit=False)

test_case_1 (__main__.TestBarBrawl) ... ok
test_case_2 (__main__.TestBarBrawl) ... ok
test_case_3 (__main__.TestBarBrawl) ... ok

----------------------------------------------------------------------
Ran 3 tests in 0.003s

OK
